# Création de l'IA détecteur de plagiat
- avant d'exécuter ce notebook, il faut éxécuter le fichier unpack.py

In [9]:
import os
from transformers import BertTokenizer, BertModel
import torch
import faiss
import pickle
import numpy as np
from sentence_transformers import util

On commence par importer toutes les librairies nécessaires pour le transformers

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# Chemins des dossiers
original = "output/source/"
copie = "output/suspicious/"


Ajouter expliations

On créé ensuite la fonction qui va générer les embeddings via BERT

In [14]:
def embed_bert(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[0, 1:-1, :].mean(dim=0).numpy()
    return embeddings

Expliquer la fonction

On génère ensuite un embedding de tout le corpus de textes origniaux

In [15]:
corpus = []
corpus_embeddings = []
for filename in os.listdir(original):
    if filename.endswith(".txt"):
        with open(os.path.join(original, filename), 'r') as f:
            texte_original = f.read()
            corpus.append(texte_original)
            corpus_embeddings.append(embed_bert(texte_original, tokenizer, model))

corpus_embeddings = np.array(corpus_embeddings)

In [22]:
def check_plagiarism(text, tokenizer, model, index, corpus_embeddings, output_file="plagiat.txt", threshold=0.8):
    
    # Calcul de l'embedding du texte à vérifier
    text_embedding = embed_bert(text, tokenizer, model)
    D, I = index.search(np.array([text_embedding]), k=5)  # Recherche des k voisins les plus proches

    # Stocker les textes similaires
    similar_texts = []

    # Ouverture du fichier pour écrire les résultats
    with open(output_file, "w", encoding="utf-8") as file:
        for i in range(len(I[0])):
            # Calcul de la similarité cosinus
            similarity = util.cos_sim(text_embedding, corpus_embeddings[I[0][i]])
            if similarity >= threshold:
                detected_text = corpus[I[0][i]]
                similar_texts.append({
                    'text': detected_text,
                    'similarity': similarity.item()
                })
                # Écriture dans le fichier
                file.write(f"Plagiat détecté :\n")
                file.write(f"- Texte source : {detected_text}\n")
                file.write(f"- Texte Similaire : {similar_texts}\n")
                file.write(f"- Similarité : {similarity.item():.2f}\n\n")

    print(f"Résultats de plagiat enregistrés dans {output_file}")
    return similar_texts

On créé maintenant l'index FAISS puis on parcours le dossier contenant les copies pour trouver les plagiats

In [23]:
# Créer l'index FAISS
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Vérifier le plagiat
for filename in os.listdir(copie):
    if filename.endswith(".txt"):
        with open(os.path.join(copie, filename), 'r') as f:
            texte_copie = f.read()
            resultats = check_plagiarism(texte_copie, tokenizer, model, index, corpus_embeddings)



Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
